In [1]:
import pandas as pd
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from konlpy.tag import Kkma
from langdetect import detect
import re
from nltk.stem import WordNetLemmatizer
from deep_translator import GoogleTranslator
from nltk.sentiment.vader import SentimentIntensityAnalyzer



In [2]:
data = pd.read_excel('油管.xlsx')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2795 entries, 0 to 2794
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   媒体ID    2795 non-null   object
 1   账号ID    2795 non-null   object
 2   账号名     2795 non-null   object
 3   粉丝量     2795 non-null   int64 
 4   标题      2795 non-null   object
 5   简介      2778 non-null   object
 6   发布日期    2795 non-null   object
 7   URL     2795 non-null   object
 8   视频时长    2795 non-null   int64 
 9   观看量     2795 non-null   int64 
 10  点赞量     2795 non-null   int64 
 11  评论量     2795 non-null   int64 
dtypes: int64(5), object(7)
memory usage: 262.2+ KB


In [5]:
# check if 媒体ID is unique
len(data['媒体ID'].unique())

2795

In [7]:
# check if 账号名 and 账号ID mean the same
data['账号名'].unique()
data['账号ID'].unique()

array(['leagueoflegends', 'T1_Faker'], dtype=object)

In [5]:
# drop no need columns
data.drop(['账号名', 'URL'], axis=1, inplace=True)


In [4]:
# check NA
data.isna().sum()

媒体ID     0
账号ID     0
粉丝量      0
标题       0
简介      17
发布日期     0
视频时长     0
观看量      0
点赞量      0
评论量      0
dtype: int64

In [6]:
# drop NA
data.dropna(inplace=True)

In [7]:
# split 发布时间 = 日期 + 时间
data['post_date'] = data['发布日期'].apply(lambda x: x.split(' ')[0])
data['post_time'] = data['发布日期'].apply(lambda x: x.split(' ')[1])

# drop 发布日期
data.drop('发布日期', axis=1, inplace = True)

In [8]:
# based on 标题, the data has to split into 2 parts: english and korean
data['language_tag'] = data['标题'].apply(lambda x: int(detect(x) == 'ko'))

In [9]:
en = data[data['language_tag'] == 0][['媒体ID', '账号ID', '标题', '简介']]
ko = data[data['language_tag'] == 1][['媒体ID', '账号ID', '标题', '简介']]

In [ ]:
translator = GoogleTranslator(source='ko', target='en')

ko['简介'] = ko['简介'].apply(lambda x : translator.translate(x))
ko['标题'] = ko['标题'].apply(lambda x : translator.translate(x))

In [10]:
# kkma = Kkma()
# ko['标题'] = ko['标题'].apply(lambda x: re.split(r'[^\w\s]', x))
# ko['简介'] = ko['简介'].apply(lambda x: re.sub(r'http\S+', '', x)).apply(lambda x: re.sub(r'[^\w\s]', '', x)).apply(lambda x: re.sub(r'\n', '', x))
# ko['word_pos_tag'] = ko['简介'].apply(lambda x: kkma.pos(re.sub(r'[^\w\s]', '', x)))


In [15]:
translated_data = pd.concat([en, ko], ignore_index=True)

In [16]:
translated_data

,媒体ID,账号ID,标题,简介
0,J_TQK480XDk,leagueoflegends,Champion Insights: Ambessa | Behind-the-scenes...,A look behind-the-scenes at how Ambessa Medard...
1,3dAb0HADFYU,leagueoflegends,Ambessa Champion Spotlight (feat. Mylon) | Gam...,"Conquer the top lane and beyond with Ambessa, ..."
2,o69dtlznmug,leagueoflegends,Ambessa: The Matriarch of War | Champion Trail...,You think yourself a wolf? Prove it. Ambessa i...
3,s5u1_XUcrSc,leagueoflegends,Jinx Fixes Everything Trailer | Gameplay - Lea...,Play as Jinx to explore your favorite haunts i...
4,R8OqqaBwcl8,leagueoflegends,Linkin Park - Heavy Is The Crown | Worlds 2024...,Watch the live performance of Heavy Is The Cro...
...,...,...,...,...
2773,bJyiMBgBTR0,T1_Faker,SKT T1 Faker Syndra / I forgot that Faker was ...,'Classic Wave' is the ID Faker used during his...
2774,cArBUMfGI9Q,T1_Faker,SKT T1 Faker Zed / KDA 12/2/6 Faker's Mid Zed!...,Here is Faker's March 10th Mid Zed Solo Rank v...
2775,lBvq9VNLJiU,T1_Faker,FAKER: TOP Fiora Faker's Fiora! This is how yo...,Faker's March 6th Top Fiora Solo Rank! Short a...
2776,nhP8-FZ0G3E,T1_Faker,SKT T1 Faker Aatrox / Patched Aatrox! Faker al...,Here is Faker's March 9 Jungle Aatrox Solo Ran...


In [28]:
# English process
# nltk.download()   --- if not run success, uncomment the line, wait until download the close the window

stop_words = set(stopwords.words('english'))

def get_clean_words(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    words = word_tokenize(text.lower())
    words = [w.strip() for w in words]
    no_stops = [w for w in words if not w in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in no_stops]
    return lemmatized_tokens

In [39]:
# create dimension tables

words_dim = translated_data[['媒体ID', '简介']]
title_dim = translated_data[['媒体ID', '标题']]
id_dim = pd.DataFrame({'id':['leagueoflegends', 'T1_Faker'], 'name':['League of Legends', 'Faker']})

In [40]:
words_dim['clean_words'] = words_dim['简介'].apply(get_clean_words).apply(lambda x: pos_tag(x))
title_dim['标题'] = title_dim['标题'].apply(lambda x: re.split(r'[^\w\s]', x))

/var/folders/jy/j2w_t12n7sxby0bnmzq56hjm0000gn/T/ipykernel_77592/3099777941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  words_dim['clean_words'] = words_dim['简介'].apply(get_clean_words).apply(lambda x: pos_tag(x))
/var/folders/jy/j2w_t12n7sxby0bnmzq56hjm0000gn/T/ipykernel_77592/3099777941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_dim['标题'] = title_dim['标题'].apply(lambda x: re.split(r'[^\w\s]', x))


In [48]:
word = pd.Series()
pos_tag = pd.Series()

for i in words_dim['clean_words']:
    w = ''
    t = ''
    for j in i:
        w+=j[0]+','
        t+=j[1]+','
    word.loc[len(word)] = w
    pos_tag.loc[len(pos_tag)] = t
    
words_dim['word'] = word
words_dim['pos_tag'] = pos_tag


In [51]:
sentiment_data = words_dim[['媒体ID', '简介']]
words_dim.drop(['简介', 'clean_words'], axis=1, inplace=True)

In [56]:
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):

    scores = analyzer.polarity_scores(text)

    sentiment = 1 if scores['pos'] > 0 else 0

    return sentiment

sentiment_data['sentiment'] = sentiment_data['简介'].apply(get_sentiment)

In [58]:
sentiment_dim = pd.DataFrame({'sentiment':[0,1], 'meaning':['negative', 'positive']})

In [52]:
data.drop(['标题', '简介', 'language_tag'], axis=1, inplace=True)

KeyError: "['标题', '简介', 'language_tag'] not found in axis"

In [12]:
# en['标题'] = en['标题'].apply(lambda x: re.sub(r'\n', '', x))

# en['words_pos_tag'] = en['简介'].apply(get_sentiment_words)

# # rebuild 简介from words

# 简介= pd.Series()

# for i in en['words_pos_tag']:
#     s = ''
#     for j in i:
#         s += j[0]+' '
#     简介.loc[len(简介)] = s.strip()

# en['简介'] = 简介


In [13]:
# en_content = en[['媒体ID', '简介']]
# ko_content = ko[['媒体ID', '简介']]

# content_dim = pd.concat([en_content, ko_content], ignore_index=True)

In [14]:
# en_title = en[['媒体ID', '标题']]
# ko_title = ko[['媒体ID', '标题']]

# title_dim = pd.concat([en_title, ko_title], ignore_index=True)

In [60]:
words_dim.to_excel('words_dim.xlsx')
data.to_excel('data.xlsx')
sentiment_data.to_excel('sentiment_data.xlsx')
id_dim.to_excel('id_dim.xlsx')
title_dim.to_excel('title_dim.xlsx')
sentiment_dim.to_excel('sentiment_dim.xlsx')